In [2]:
import pandas as pd
from math import comb
from ax.service.ax_client import AxClient
import time
import sys

sys.path.append('../../py_files')
from sdlnano import BayesianOptimizer_Auto
from calculation import BatchCalculation

In [ ]:
# Design space size calculation


def calculate_combinations(num_variables, total_sum, step):
    steps = int(total_sum / step)
    return comb(steps + num_variables - 1, num_variables - 1)

# Define parameters
step = 0.05
total_sum = 1

# Calculate combinations for each group
count_group1 = calculate_combinations(10, total_sum, step)  # 3 Solid lipids + 3 Polymers + 3 Liquid lipids + 1 Drug
count_group2 = calculate_combinations(4, total_sum, step)   # 3 Surfactants + 1 Water

# Calculate total combinations
total_combinations = count_group1 * count_group2


print("organic design space size: ", round(count_group1/(1e6),2), 'Million')
print("aqueous design space size: ", round(count_group2/(1e3),2), 'Thousand')
print("design space size: ", round(total_combinations/(1e9),2), 'Billion')


organic design space size:  10.02 Million
aqueous design space size:  1.77 Thousand
design space size:  17.74 Billion


# Check the iteration number

In [4]:
iteration = BayesianOptimizer_Auto.get_iteration_number()
print("This is the iteration: #", iteration)

This is the iteration: # 7


# Check the drug

In [5]:
drug = 'ACE'
print("This is the drug:", drug)

This is the drug: ACE


# Optimizer initialization (N/A)

# Generate recommendations

In [6]:
time_start = time.time()

previous_optimizer_file_name = '../iteration_' + str(iteration-1) + '/optimizer/optimizer_load_' + str(iteration-1) + '.json'
# generate recomms
ax_client_init = AxClient.load_from_json_file(previous_optimizer_file_name)
unlabeled_trial, ax_client_design = BayesianOptimizer_Auto.generate_trials(ax_client=ax_client_init, num_of_trials = 16, drug = drug, bopt=1)

time_end = time.time()

print(round((time_end-time_start)/60), 'min')

[INFO 02-12 10:44:54] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 02-12 10:45:37] ax.service.ax_client: Generated new trial 112 with parameters {'Drug_MW': 0.354, 'Drug_LogP': 0.391, 'Drug_TPSA': 0.067, 'Drug': 0.943058, 'SL_1': 0.0, 'SL_2': 0.0, 'SL_3': 0.0, 'LL_1': 0.0, 'LL_2': 1.0, 'LL_3': 0.0, 'P_1': 0.010272, 'P_2': 0.995575, 'P_3': 0.69511, 'S_1': 0.0, 'S_2': 0.965023, 'S_3': 0.939447, 'Water': 1.0} using model BoTorch.
/Users/zeqing/opt/anaconda3/envs/sdlnano/lib/python3.10/site-packages/ax/core/data.py:288: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return cls(df=pd.con

9 min


In [ ]:
# process the trails into ratios

unlabeled_trial_processed = BayesianOptimizer_Auto.process_trails(unlabeled_trial)
unlabeled_trial_processed.describe()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.0,16.00,16.0,16.000000,16.000000,16.0,16.0,16.000000,16.000000,16.0,16.000000,16.000000,16.000000,16.0,16.000000,16.000000,16.000000
mean,119.500000,354.0,3.91,67.0,0.297750,0.071125,0.0,0.0,0.062625,0.193750,0.0,0.007687,0.098250,0.268625,0.0,0.504625,0.203437,0.291875
std,4.760952,0.0,0.00,0.0,0.095205,0.094401,0.0,0.0,0.097003,0.064698,0.0,0.017373,0.097189,0.089240,0.0,0.141214,0.241081,0.270859
min,112.000000,354.0,3.91,67.0,0.146000,0.000000,0.0,0.0,0.000000,0.089000,0.0,0.000000,0.000000,0.136000,0.0,0.332000,0.000000,0.000000
25%,115.750000,354.0,3.91,67.0,0.228250,0.000000,0.0,0.0,0.000000,0.142500,0.0,0.000000,0.000000,0.190750,0.0,0.452250,0.000000,0.000000
50%,119.500000,354.0,3.91,67.0,0.259000,0.014500,0.0,0.0,0.000000,0.198000,0.0,0.000000,0.114000,0.264500,0.0,0.500000,0.002000,0.422000
75%,123.250000,354.0,3.91,67.0,0.374250,0.131000,0.0,0.0,0.133500,0.255750,0.0,0.004250,0.167000,0.342750,0.0,0.506500,0.483750,0.543750
max,127.000000,354.0,3.91,67.0,0.472000,0.287000,0.0,0.0,0.254000,0.274000,0.0,0.058000,0.273000,0.390000,0.0,1.000000,0.500000,0.583000


In [8]:
unlabeled_trial
unlabeled_trial.describe()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000,16.000,16.000,16.000000,16.000000,16.0,16.0,16.000000,16.000000,16.0,16.000000,16.000000,16.000000,16.0,16.000000,16.000000,16.000000
mean,119.500000,0.354,0.391,0.067,0.915395,0.214894,0.0,0.0,0.259570,0.663835,0.0,0.024198,0.377872,0.820142,0.0,0.924519,0.416960,0.562500
std,4.760952,0.000,0.000,0.000,0.126861,0.295780,0.0,0.0,0.408040,0.318679,0.0,0.052552,0.378877,0.123865,0.0,0.101372,0.488119,0.512348
min,112.000000,0.354,0.391,0.067,0.572823,0.000000,0.0,0.0,0.000000,0.206535,0.0,0.000000,0.000000,0.637712,0.0,0.715993,0.000000,0.000000
25%,115.750000,0.354,0.391,0.067,0.875804,0.000000,0.0,0.0,0.000000,0.388256,0.0,0.000000,0.000000,0.736411,0.0,0.839838,0.000000,0.000000
50%,119.500000,0.354,0.391,0.067,0.977232,0.035730,0.0,0.0,0.000000,0.663272,0.0,0.000000,0.447665,0.787905,0.0,1.000000,0.003665,1.000000
75%,123.250000,0.354,0.391,0.067,1.000000,0.320960,0.0,0.0,0.512461,0.996917,0.0,0.014852,0.632445,0.935588,0.0,1.000000,0.946454,1.000000
max,127.000000,0.354,0.391,0.067,1.000000,0.966720,0.0,0.0,1.000000,1.000000,0.0,0.169392,1.000000,1.000000,0.0,1.000000,1.000000,1.000000


In [ ]:
# Convert ratios to volumes

transfer = BatchCalculation.converter(unlabeled_trial_processed)
transfer.describe()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Solvent
count,16.000000,16.0,16.00,16.0,16.000000,16.000000,16.0,16.0,16.000000,16.000000,16.0,16.000000,16.000000,16.000000,16.0,16.000000,16.000000,16.00000,16.000000
mean,119.500000,354.0,3.91,67.0,104.212500,24.893750,0.0,0.0,21.918750,67.812500,0.0,2.690625,34.387500,94.018750,0.0,504.625000,203.437500,291.87500,0.065625
std,4.760952,0.0,0.00,0.0,33.321887,33.040414,0.0,0.0,33.951052,22.644216,0.0,6.080631,34.016087,31.234142,0.0,141.213727,241.080614,270.85885,0.190367
min,112.000000,354.0,3.91,67.0,51.100000,0.000000,0.0,0.0,0.000000,31.150000,0.0,0.000000,0.000000,47.600000,0.0,332.000000,0.000000,0.00000,-0.350000
25%,115.750000,354.0,3.91,67.0,79.887500,0.000000,0.0,0.0,0.000000,49.875000,0.0,0.000000,0.000000,66.762500,0.0,452.250000,0.000000,0.00000,0.000000
50%,119.500000,354.0,3.91,67.0,90.650000,5.075000,0.0,0.0,0.000000,69.300000,0.0,0.000000,39.900000,92.575000,0.0,500.000000,2.000000,422.00000,0.000000
75%,123.250000,354.0,3.91,67.0,130.987500,45.850000,0.0,0.0,46.725000,89.512500,0.0,1.487500,58.450000,119.962500,0.0,506.500000,483.750000,543.75000,0.087500
max,127.000000,354.0,3.91,67.0,165.200000,100.450000,0.0,0.0,88.900000,95.900000,0.0,20.300000,95.550000,136.500000,0.0,1000.000000,500.000000,583.00000,0.350000


In [10]:
#dead_volume = 3.0 # mL

round(transfer[['Drug','SL_1', 'SL_2', 'SL_3', 'LL_1', 'LL_2', 'LL_3', 'P_1', 'P_2', 'P_3', 'S_1', 'S_2', 'S_3', 'Water', 'Solvent']].sum()/1000,1)

Drug       1.7
SL_1       0.4
SL_2       0.0
SL_3       0.0
LL_1       0.4
LL_2       1.1
LL_3       0.0
P_1        0.0
P_2        0.6
P_3        1.5
S_1        0.0
S_2        8.1
S_3        3.3
Water      4.7
Solvent    0.0
dtype: float64

In [11]:
# file path
ax_client_design_path = "optimizer/optimizer_design_" + str(iteration) + ".json"

unlabeled_trial_path = "data/unlabeled_" + str(iteration) + ".xlsx"

unlabeled_trial_processed_path = "data/unlabeled_processed_" + str(iteration) + ".xlsx"

transfer_path = 'data/transfer_' + str(iteration) + '.xlsx'


In [12]:
ax_client_design.save_to_json_file(ax_client_design_path)

unlabeled_trial.to_excel(unlabeled_trial_path, index=False)

unlabeled_trial_processed.to_excel(unlabeled_trial_processed_path, index=False)

transfer.drop(['Drug_MW', 'Drug_LogP', 'Drug_TPSA'], axis=1).to_excel(transfer_path, index = False)

[INFO 02-12 10:54:01] ax.service.ax_client: Saved JSON-serialized state of optimization to `optimizer/optimizer_design_7.json`.


# Generate new protocol

In [13]:
# Paths
template_protocol = '../../templates/OT_protocol_template.py'

output_script_path = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_.py'


In [14]:
# Update the script with Excel data
updated_script = BatchCalculation.update_transfer_script(template_protocol, transfer_path, output_script_path) 


In [15]:
updated_script

'protocol/iteration_7_OT_2_protocol_.py'

# Perform experiment

# Results

In [16]:
results = BatchCalculation.parameters_and_complexity(iteration)
results

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Complexity,Complexity_STD
0,112,354,3.91,67,0.259,0.000,0,0,0.000,0.274,0,0.003,0.273,0.191,0,0.332,0.323,0.344,6,0
1,113,354,3.91,67,0.250,0.000,0,0,0.129,0.261,0,0.000,0.165,0.195,0,0.506,0.494,0.000,6,0
2,114,354,3.91,67,0.219,0.000,0,0,0.147,0.252,0,0.044,0.166,0.172,0,0.534,0.466,0.000,7,0
3,115,354,3.91,67,0.362,0.143,0,0,0.000,0.145,0,0.010,0.000,0.339,0,0.457,0.000,0.543,5,0
4,116,354,3.91,67,0.234,0.000,0,0,0.234,0.208,0,0.000,0.144,0.179,0,0.508,0.492,0.000,6,0
5,117,354,3.91,67,0.259,0.079,0,0,0.024,0.270,0,0.000,0.170,0.197,0,0.505,0.495,0.000,7,0
6,118,354,3.91,67,0.146,0.000,0,0,0.254,0.254,0,0.000,0.156,0.190,0,0.500,0.500,0.000,6,0
7,119,354,3.91,67,0.411,0.029,0,0,0.000,0.188,0,0.000,0.000,0.372,0,0.500,0.000,0.500,4,0
8,120,354,3.91,67,0.229,0.066,0,0,0.000,0.273,0,0.000,0.200,0.232,0,0.519,0.481,0.000,6,0
9,121,354,3.91,67,0.214,0.000,0,0,0.214,0.214,0,0.008,0.214,0.136,0,1.000,0.000,0.000,6,0


In [17]:
size_raw = BatchCalculation.size_raw(iteration)
size_raw

,Data Quality,Item,Size,PD Index
0,Good,A1,222.887653,0.189674
1,Good,A2,216.078907,0.203067
2,Good,A3,221.519243,0.154002
3,Good,A4,227.818205,0.176828
4,Good,A5,226.458207,0.227162
5,Good,A6,213.634757,0.207547
6,Good,B1,212.518050,0.227852
7,Good,B2,218.544279,0.189779
8,Good,B3,218.209661,0.206452
9,Good,B4,223.573956,0.299811


In [18]:
size_processed = BatchCalculation.process_formulations(size_raw)
size_processed

,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,Formulation 1,220.161934,3.601593,0.182248,0.025362,1
1,Formulation 2,222.637056,7.825819,0.203845,0.025370,1
2,Formulation 3,216.423997,3.386784,0.208028,0.019086,1
3,Formulation 4,215.402571,10.847991,0.290551,0.016527,1
4,Formulation 5,200.077728,1.585732,0.181269,0.017931,1
5,Formulation 6,221.858110,0.120722,0.189454,0.017035,1
6,Formulation 7,209.754216,7.467246,0.203273,0.022909,1
7,Formulation 8,187.736471,7.267564,0.183646,0.021559,1
8,Formulation 9,213.460331,9.798711,0.208274,0.029466,1
9,Formulation 10,195.384545,2.122183,0.193322,0.027381,1


In [19]:
results_final = pd.concat([results, size_processed], axis=1)
results_final

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,...,S_3,Water,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,112,354,3.91,67,0.259,0.000,0,0,0.000,0.274,...,0.323,0.344,6,0,Formulation 1,220.161934,3.601593,0.182248,0.025362,1
1,113,354,3.91,67,0.250,0.000,0,0,0.129,0.261,...,0.494,0.000,6,0,Formulation 2,222.637056,7.825819,0.203845,0.025370,1
2,114,354,3.91,67,0.219,0.000,0,0,0.147,0.252,...,0.466,0.000,7,0,Formulation 3,216.423997,3.386784,0.208028,0.019086,1
3,115,354,3.91,67,0.362,0.143,0,0,0.000,0.145,...,0.000,0.543,5,0,Formulation 4,215.402571,10.847991,0.290551,0.016527,1
4,116,354,3.91,67,0.234,0.000,0,0,0.234,0.208,...,0.492,0.000,6,0,Formulation 5,200.077728,1.585732,0.181269,0.017931,1
5,117,354,3.91,67,0.259,0.079,0,0,0.024,0.270,...,0.495,0.000,7,0,Formulation 6,221.858110,0.120722,0.189454,0.017035,1
6,118,354,3.91,67,0.146,0.000,0,0,0.254,0.254,...,0.500,0.000,6,0,Formulation 7,209.754216,7.467246,0.203273,0.022909,1
7,119,354,3.91,67,0.411,0.029,0,0,0.000,0.188,...,0.000,0.500,4,0,Formulation 8,187.736471,7.267564,0.183646,0.021559,1
8,120,354,3.91,67,0.229,0.066,0,0,0.000,0.273,...,0.481,0.000,6,0,Formulation 9,213.460331,9.798711,0.208274,0.029466,1
9,121,354,3.91,67,0.214,0.000,0,0,0.214,0.214,...,0.000,0.000,6,0,Formulation 10,195.384545,2.122183,0.193322,0.027381,1


In [20]:
# µg/mL
results_final['Solu'] = results_final['Drug'] * 600 / 0.3
results_final['Solu_STD'] = 0

results_final.loc[results_final['Formulation Quality'] == 0, 'Solu'] = 0
results_final

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,112,354,3.91,67,0.259,0.000,0,0,0.000,0.274,...,6,0,Formulation 1,220.161934,3.601593,0.182248,0.025362,1,518.0,0
1,113,354,3.91,67,0.250,0.000,0,0,0.129,0.261,...,6,0,Formulation 2,222.637056,7.825819,0.203845,0.025370,1,500.0,0
2,114,354,3.91,67,0.219,0.000,0,0,0.147,0.252,...,7,0,Formulation 3,216.423997,3.386784,0.208028,0.019086,1,438.0,0
3,115,354,3.91,67,0.362,0.143,0,0,0.000,0.145,...,5,0,Formulation 4,215.402571,10.847991,0.290551,0.016527,1,724.0,0
4,116,354,3.91,67,0.234,0.000,0,0,0.234,0.208,...,6,0,Formulation 5,200.077728,1.585732,0.181269,0.017931,1,468.0,0
5,117,354,3.91,67,0.259,0.079,0,0,0.024,0.270,...,7,0,Formulation 6,221.858110,0.120722,0.189454,0.017035,1,518.0,0
6,118,354,3.91,67,0.146,0.000,0,0,0.254,0.254,...,6,0,Formulation 7,209.754216,7.467246,0.203273,0.022909,1,292.0,0
7,119,354,3.91,67,0.411,0.029,0,0,0.000,0.188,...,4,0,Formulation 8,187.736471,7.267564,0.183646,0.021559,1,822.0,0
8,120,354,3.91,67,0.229,0.066,0,0,0.000,0.273,...,6,0,Formulation 9,213.460331,9.798711,0.208274,0.029466,1,458.0,0
9,121,354,3.91,67,0.214,0.000,0,0,0.214,0.214,...,6,0,Formulation 10,195.384545,2.122183,0.193322,0.027381,1,428.0,0


In [21]:
results_normalized = BayesianOptimizer_Auto.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.220162,0.003602,0.259,0.0,0.182248,0.025362,0.500000
1,1,0.222637,0.007826,0.250,0.0,0.203845,0.025370,0.500000
2,1,0.216424,0.003387,0.219,0.0,0.208028,0.019086,0.583333
3,1,0.215403,0.010848,0.362,0.0,0.290551,0.016527,0.416667
4,1,0.200078,0.001586,0.234,0.0,0.181269,0.017931,0.500000
5,1,0.221858,0.000121,0.259,0.0,0.189454,0.017035,0.583333
6,1,0.209754,0.007467,0.146,0.0,0.203273,0.022909,0.500000
7,1,0.187736,0.007268,0.411,0.0,0.183646,0.021559,0.333333
8,1,0.213460,0.009799,0.229,0.0,0.208274,0.029466,0.500000
9,1,0.195385,0.002122,0.214,0.0,0.193322,0.027381,0.500000


In [22]:
results_normalized = BayesianOptimizer_Auto.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.220162,0.003602,0.259,0.0,0.182248,0.025362,0.500000
1,1,0.222637,0.007826,0.250,0.0,0.203845,0.025370,0.500000
2,1,0.216424,0.003387,0.219,0.0,0.208028,0.019086,0.583333
3,1,0.215403,0.010848,0.362,0.0,0.290551,0.016527,0.416667
4,1,0.200078,0.001586,0.234,0.0,0.181269,0.017931,0.500000
5,1,0.221858,0.000121,0.259,0.0,0.189454,0.017035,0.583333
6,1,0.209754,0.007467,0.146,0.0,0.203273,0.022909,0.500000
7,1,0.187736,0.007268,0.411,0.0,0.183646,0.021559,0.333333
8,1,0.213460,0.009799,0.229,0.0,0.208274,0.029466,0.500000
9,1,0.195385,0.002122,0.214,0.0,0.193322,0.027381,0.500000


In [23]:
labeled_data_path = 'data/labeled_' + str(iteration) + '.xlsx'
labeled_norm_path = 'data/labeled_norm_' + str(iteration) + '.xlsx'


In [24]:
results_final.to_excel(labeled_data_path, index=False)
results_normalized.to_excel(labeled_norm_path, index=False)

# Load the labeled data to update the optimizer

In [25]:
labeled_norm = pd.read_excel(labeled_norm_path)
labeled_norm.head()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,112,354,3.91,67,0.259,0.000,0,0,0.000,0.274,...,0.500000,0,Formulation 1,0.220162,0.003602,0.182248,0.025362,1,0.259,0
1,113,354,3.91,67,0.250,0.000,0,0,0.129,0.261,...,0.500000,0,Formulation 2,0.222637,0.007826,0.203845,0.025370,1,0.250,0
2,114,354,3.91,67,0.219,0.000,0,0,0.147,0.252,...,0.583333,0,Formulation 3,0.216424,0.003387,0.208028,0.019086,1,0.219,0
3,115,354,3.91,67,0.362,0.143,0,0,0.000,0.145,...,0.416667,0,Formulation 4,0.215403,0.010848,0.290551,0.016527,1,0.362,0
4,116,354,3.91,67,0.234,0.000,0,0,0.234,0.208,...,0.500000,0,Formulation 5,0.200078,0.001586,0.181269,0.017931,1,0.234,0


In [26]:
ax_client_design = AxClient.load_from_json_file(ax_client_design_path)


[INFO 02-12 12:48:18] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.


In [27]:
loaded_optimizer_path = "optimizer/optimizer_load_" + str(iteration) + ".json"

In [28]:
loaded_ax_client = BayesianOptimizer_Auto.load_labeled_data(ax_client_design, labeled_norm_path)
loaded_ax_client.save_to_json_file(loaded_optimizer_path)

[INFO 02-12 12:48:19] ax.service.ax_client: Completed trial 112 with data: {'Solu': (0.259, 0), 'Size': (0.220162, 0.003602), 'PDI': (0.182248, 0.025362), 'Complexity': (0.5, 0)}.
[INFO 02-12 12:48:19] ax.service.ax_client: Completed trial 113 with data: {'Solu': (0.25, 0), 'Size': (0.222637, 0.007826), 'PDI': (0.203845, 0.02537), 'Complexity': (0.5, 0)}.
[INFO 02-12 12:48:19] ax.service.ax_client: Completed trial 114 with data: {'Solu': (0.219, 0), 'Size': (0.216424, 0.003387), 'PDI': (0.208028, 0.019086), 'Complexity': (0.583333, 0)}.
[INFO 02-12 12:48:19] ax.service.ax_client: Completed trial 115 with data: {'Solu': (0.362, 0), 'Size': (0.215403, 0.010848), 'PDI': (0.290551, 0.016527), 'Complexity': (0.416667, 0)}.
[INFO 02-12 12:48:19] ax.service.ax_client: Completed trial 116 with data: {'Solu': (0.234, 0), 'Size': (0.200078, 0.001586), 'PDI': (0.181269, 0.017931), 'Complexity': (0.5, 0)}.
[INFO 02-12 12:48:19] ax.service.ax_client: Completed trial 117 with data: {'Solu': (0.259, 